In [108]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [109]:
pages = range(2,25)
url = f"https://twitchtracker.com/channels/viewership?page=1"
def parse_html(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Platform; Security; OS-or-CPU; Localization; rv:1.4) Gecko/20030624 Netscape/7.1 (ax)'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    df=pd.read_html(str(soup), flavor='bs4')[0]
    return df

In [111]:
url = "https://twitchtracker.com/channels/viewership?page="
def get_data(url, pages):
    import time
    t=5
    df = parse_html(f'{url}=1') # getting the first page
    pages = range(2,pages)
    for page in pages: # getting all other pages, defined by argument
        print(page)
        new_df = parse_html(f"{url}{page}")
        df = df.append(new_df, ignore_index=True)
        time.sleep(t)
    return df

df = get_data(url, 25)

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [112]:
def streamers_df_cleaning(df):
    df = df[df["Time Streamed"].str.contains("ads")==False]
    # creating a new column that contains hours where we replace all hours with an emptry string and convert numbers in string to float
    df['Time Streamed (in hours)'] = df['Time Streamed'].str.replace('hours','').apply(lambda x: float(x))


    # do same thing for hours watched
    df['Hours Watched'] = df['Hours Watched'].apply(lambda x: int(x))

    # do it for every other columns
    df['AVG Viewers'] = df['AVG Viewers'].apply(lambda x: int(x))
    df['All Time Peak Viewers'] = df['All Time Peak Viewers'].apply(lambda x: int(x))
    df['Total Followers'] = df['Total Followers'].apply(lambda x: int(x))
    df['Total Views'] = df['Total Views'].apply(lambda x: int(x))
    df['Followers Gained'] = df['Followers Gained'].str.replace('+','').apply(lambda x: int(x))
    # check if it worked
    df.dtypes

    # drop and rename columns
    df.drop(columns=['Unnamed: 0', 'Unnamed: 1', 'Time Streamed'], axis=1,inplace=True)
    df.rename(columns={'Unnamed: 2':'username'}, inplace=True)

    # sort according to importance
    df = df[['username', 'AVG Viewers', 'Time Streamed (in hours)', 'Hours Watched', 'Followers Gained', 'Total Followers', 'Total Views', 'All Time Peak Viewers']]
    return df

In [113]:
clean_streamers_df = streames_df_cleaning(df)

<ipython-input-99-59a875cf49c2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Time Streamed (in hours)'] = df['Time Streamed'].str.replace('hours','').apply(lambda x: float(x))
<ipython-input-99-59a875cf49c2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Hours Watched'] = df['Hours Watched'].apply(lambda x: int(x))
<ipython-input-99-59a875cf49c2>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [131]:
#get df for every broadcast //done

def parse_games_2(user_name):
    user_name.lower()
    headers = {'User-Agent': 'Mozilla/5.0 (Platform; Security; OS-or-CPU; Localization; rv:1.4) Gecko/20030624 Netscape/7.1 (ax)'}
    response = requests.get(url = f"https://twitchtracker.com/{user_name}/games", headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    df=pd.read_html(str(soup))[0]
    return df

In [142]:
quack=parse_games_2('Quackity')

In [143]:
quack.head()

,#,Game,Avg. viewers,Max. viewers,Followers,Total airtime,Last seen
0,#1,Minecraft,32652,381550,2601.1 /hour,13355 34.2%,2021-06-02 21:20:00
1,#2,Just Chatting,18207,230817,1555.4 /hour,9447 24.2%,2021-04-23 01:30:00
2,#3,Toontown Online,1524,4299,236 /hour,2032 5.2%,2020-06-19 07:20:00
3,#4,Jackbox Party Packs,126620,232299,5230.6 /hour,1376 3.5%,2021-04-15 03:40:00
4,#5,Among Us,20987,109720,2384.7 /hour,1254 3.2%,2020-12-30 04:30:00


In [153]:
def clean_gamer_df(username):
    df=parse_games_2(username)
    df["Total airtime"] = df["Total airtime"].astype(str).str[:-1]
    df[['Followers per hour', 'to_remove']]=df['Followers'].str.split(' /',expand=True)
    df = df.drop(columns=['Followers', 'to_remove'])
    df[['Total airtime','Percentage of total airtime']]=df['Total airtime'].str.split(' ',expand=True) 
    df['Percentage of total airtime'] = df['Percentage of total airtime'].apply(lambda x: float(x))
    df['Followers per hour'] = df['Followers per hour'].apply(lambda x: float(x))
    df['Last seen'] = pd.to_datetime(df['Last seen'])
    df=df.rename(columns={'#':'id'})
    df['id']=df['id'].astype(str).str[0]
    df=df.set_index('id')
    return df

In [154]:
clean_gamer_df('Quackity')

,Game,Avg. viewers,Max. viewers,Total airtime,Last seen,Followers per hour,Percentage of total airtime
id,,,,,,,
#,Minecraft,32652,381550,13355,2021-06-02 21:20:00,2601.1,34.2
#,Just Chatting,18207,230817,9447,2021-04-23 01:30:00,1555.4,24.2
#,Toontown Online,1524,4299,2032,2020-06-19 07:20:00,236.0,5.2
#,Jackbox Party Packs,126620,232299,1376,2021-04-15 03:40:00,5230.6,3.5
#,Among Us,20987,109720,1254,2020-12-30 04:30:00,2384.7,3.2
#,Roblox,76616,202526,1223,2021-03-22 00:10:00,4208.7,3.1
#,Habbo,18714,117637,1048,2021-01-17 00:20:00,1244.9,2.7
#,Animal Crossing: New Horizons,3412,4208,942,2020-04-20 04:40:00,557.2,2.4
#,The Sims 4,19403,164457,917,2021-03-27 02:30:00,1014.3,2.3
